In [1]:
import torch 
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
class CNN(nn.Module):
    def __init__(self,in_channels = 1, num_class =10):
        super(CNN ,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1 , out_channels=8 , kernel_size=(3,3) , stride=(1,1) , padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2) , stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8 , out_channels=16 , kernel_size=(3,3), stride=(1,1) , padding=(1,1))
        self.fc1 = nn.Linear(16*7*7 , num_class)

    def forward(self , x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x= F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0] , -1)
        x = self.fc1(x)
        return x


In [4]:
device = torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
in_channels = 1
num_class = 10
learning_rate = 0.001
batch_size = 64
num_eppochs = 5

In [6]:
train_datasets = datasets.MNIST(root ='dataset/' , train= True ,transform = transforms.ToTensor() , download=True)
train_lader = DataLoader(train_datasets , batch_size= batch_size ,shuffle=True)
test_dataset = datasets.MNIST(root ='dataset/' , train= False ,transform = transforms.ToTensor() , download = True)
teat_dataloader = DataLoader(test_dataset , batch_size=batch_size , shuffle=True)


In [7]:
model = CNN().to(device=device)


In [8]:
critision = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [9]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [10]:
for epochs in range(num_eppochs):
    for batch_idx ,(data ,targets) in enumerate(train_lader):
        data = data.to(device = device)
        targets = targets.to(device = device)
        scores = model(data)
        loss = critision(scores,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [11]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("accuracy for train dataset")
    else:
        print("accuracy o test datsets")
    num_correct = 0
    num_samples=0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device = device)
            scores = model(x)
            _,predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
        print(f'got {num_correct}/{num_samples} with accuraccy  {float(num_correct)/float(num_samples)*100:.2f}'
        )
    model.train()


In [13]:
check_accuracy(train_lader , model)

accuracy for train dataset
got 57634/60000 with accuraccy  96.06


In [14]:
check_accuracy(teat_dataloader ,model)

accuracy o test datsets
got 9633/10000 with accuraccy  96.33
